In [ ]:
import pandas as pd
import numpy as np
import torch
import gc
import matplotlib.pyplot as plt
import extrametrics as em
from neuralforecast.core import NeuralForecast
from neuralforecast.losses.pytorch import MSE, MAE, MAPE, RMSE
from neuralforecast.losses.numpy import mse, mae, mape, rmse
from neuralforecast.models import StemGNN
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import os

In [ ]:
Y_df = pd.read_csv('m5_stemgnn.csv')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])
Y_df.head()

In [ ]:
minmax_scaler = MinMaxScaler()

# Step 2: Group by unique_id to normalize each product's timeseries individually
def apply_minmax(group):
    # MinMax scaling
    group['y_minmax'] = minmax_scaler.fit_transform(group[['y']])
    return group


# Apply MinMax normalization
Y_df_minmax = Y_df.groupby('unique_id', group_keys=False).apply(apply_minmax)


# Drop the original 'y' column and rename the normalized columns
Y_df_minmax = Y_df_minmax.drop(columns=['y']).rename(columns={'y_minmax': 'y'})

print(Y_df_minmax.head())


In [ ]:
# We make validation and test splits
n_time = len(Y_df.ds.unique())
val_size = int(.2 * n_time)
test_size = int(.1 * n_time)
timeseries_count = len(Y_df.unique_id.unique())

n_time, val_size, test_size, timeseries_count

In [ ]:
# Define the parameter grid for grid search
param_grid = {
    'horizon': [13],
    'input_size': [13],
    'max_steps': [100],
    'learning_rate': [1e-2, 1e-3],
    'batch_size': [8, 16]
}

# Create a grid of parameters to search
grid = ParameterGrid(param_grid)


In [ ]:
def metrics_eval_grid(model, Y_hat_df):
    mae_model = mae(Y_hat_df['y'], Y_hat_df[f'{model}'])
    mse_model = mse(Y_hat_df['y'], Y_hat_df[f'{model}'])
    mape_model = mape(Y_hat_df['y'], Y_hat_df[f'{model}'])
    rmse_model = rmse(Y_hat_df['y'], Y_hat_df[f'{model}'])
    wmape_model = em.wmape(Y_hat_df['y'], Y_hat_df[f'{model}'])
    r_squared_model = em.r_squared(Y_hat_df['y'], Y_hat_df[f'{model}'])

    return mae_model, mse_model, rmse_model, mape_model, wmape_model, r_squared_model


In [ ]:
# Initialize an empty DataFrame to store results and a variable to track the best MAE
results_df = pd.DataFrame(columns=[
    'horizon', 'input_size', 'max_steps', 'learning_rate', 'batch_size',
    'MAE', 'MSE', 'RMSE', 'MAPE', 'WMAPE', 'R_Squared'
])
best_mae = float('inf')  # Set to infinity initially to ensure the first model is saved
best_model_path = 'best_model_weights.pth'

# Loop through each parameter combination in the grid
for params in grid:
    print(params)
    print('\n\n')
    try:
        # Create model instance with current parameters
        model = StemGNN(
            h=params['horizon'],
            input_size=params['input_size'],
            n_series=timeseries_count,
            scaler_type='robust',
            max_steps=params['max_steps'],
            early_stop_patience_steps=-1,
            val_check_steps=1,
            learning_rate=params['learning_rate'],
            loss=MAE(),
            valid_loss=None,
            batch_size=params['batch_size'],
            random_seed=1
        )

        # Create NeuralForecast instance and perform cross-validation
        nf_grid = NeuralForecast(models=[model], freq='W')
        Y_hat_df_grid = nf_grid.cross_validation(df=Y_df_minmax, val_size=val_size, test_size=test_size, n_windows=None)
        Y_hat_df_grid = Y_hat_df_grid.reset_index() if Y_hat_df_grid is not None else None
        print(Y_hat_df_grid)
        if Y_hat_df_grid is not None:
            # Compute metrics
            y_mae, y_mse, y_rmse, y_mape, y_wmape, y_r_squared = metrics_eval_grid(model, Y_hat_df_grid)

            # Add the metrics and parameters to the results_df DataFrame
            results_df = pd.concat([results_df, pd.DataFrame([{
                'horizon': params['horizon'],
                'input_size': params['input_size'],
                'max_steps': params['max_steps'],
                'learning_rate': params['learning_rate'],
                'batch_size': params['batch_size'],
                'MAE': y_mae,
                'MSE': y_mse,
                'RMSE': y_rmse,
                'MAPE': y_mape,
                'WMAPE': y_wmape,
                'R_Squared': y_r_squared
            }])], ignore_index=True)

            # Check if this model's MAE is better than the best MAE so far
            if y_mae < best_mae:
                best_mae = y_mae  # Update the best MAE
                print(f"New best model found with MAE: {y_mae:.3f}. Saved model to {best_model_path}.")
            else:
                print(f"Model with MAE: {y_mae:.3f} did not improve the best MAE: {best_mae:.3f}. Skipping save.")

            print(f"Parameters: {params}")
            print(f"MAE: {y_mae:.3f}, MSE: {y_mse:.3f}, RMSE: {y_rmse:.3f}, "
                  f"MAPE: {y_mape:.3f}%, WMAPE: {y_wmape:.3f}%, R_Squared: {y_r_squared:.3f}%\n")
        else:
            print(f"Skipping parameter combination {params} due to None predictions.\n")

    finally:
      # Check if variables exist before deleting them
      if 'model' in locals():
          del model
      if 'nf_grid' in locals():
          del nf_grid
      if 'Y_hat_df_grid' in locals():
          del Y_hat_df_grid
      gc.collect()
      torch.cuda.empty_cache()

# Display the resulting DataFrame
print(results_df)


In [ ]:
# Sort the DataFrame by MAE, MSE, RMSE, and MAPE in ascending order
results_df2 = results_df.sort_values(by=['MAE', 'MSE', 'RMSE', 'MAPE'], ascending=True)

# If you want to reset the index after sorting
results_df2 = results_df2.reset_index(drop=True)

# Save the results DataFrame to a CSV file
results_df2.to_csv('best_model_results.csv', index=False)  # index=False to omit the index column

# Optionally, print the sorted DataFrame to check the result
results_df2
